In [ ]:
# https://www.kaggle.com/c/vacancy-classification-sf02

In [10]:
import pandas as pd
import nltk
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('train.csv', sep='\t', index_col='id')
test = pd.read_csv('test.csv', sep='\t', index_col='id')
train.head()

,name,description,target
id,,,
0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [3]:
train.loc[:, 'text'] = train['name']+' '+train['description']
test.loc[:, 'text'] = test['name']+' '+test['description']
train['text'][:5]

id
0    Заведующий отделом/секцией в магазин YORK (Уру...
1    Наладчик станков и манипуляторов с ПУ Обязанно...
2    Разработчик С++ (Криптограф) <strong>Требовани...
3    Фрезеровщик <p>Условия:</p> <ul> <li>На работу...
4    Мерчендайзер/продавец-консультант <p><strong>К...
Name: text, dtype: object

In [6]:
train_wo_tags = []
for t in train['text']:
    train_wo_tags.append(
        BeautifulSoup(t, 'lxml').text
    )

test_wo_tags = []
for t in test['text']:
    test_wo_tags.append(
        BeautifulSoup(t, 'lxml').text
    )

In [9]:
X_train, X_test, y_train, y_test = train_test_split(train_wo_tags, train['target'], test_size=0.3, shuffle=True, random_state=42)

stop_words = nltk.corpus.stopwords.words('russian')

model = Pipeline([
    ('vect', TfidfVectorizer(stop_words=stop_words,
                             ngram_range=(1,1),
                             max_df=0.95, min_df=5)),
    ('clf', LogisticRegression(random_state=42)),
])

model.fit(X_train, y_train)

roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

0.9912961105739081

In [14]:
model2 = Pipeline([
    ('vect', CountVectorizer(stop_words=stop_words, 
                             ngram_range=(1,1))),
    ('clf', LogisticRegression(random_state=42)),
])

model2.fit(X_train, y_train)

roc_auc_score(y_test, model2.predict_proba(X_test)[:,1])

0.9902108564314233

In [ ]:
submission = pd.DataFrame()
submission.loc[:,'id'] = test.reset_index()['id']
submission.loc[:,'target'] = model.predict_proba(test_wo_tags)[:, 1]
submission.to_csv('submission.csv', sep=',', index=False, encoding='utf-8')

In [30]:
# TODO

from vecstack import stacking

vect = TfidfVectorizer(stop_words=stop_words,
                             ngram_range=(1,1),
                             max_df=0.95, min_df=5)

x_train = vect.fit_transform(X_train)
x_test = vect.transform(X_test)

models = [
    RandomForestClassifier(random_state=42),
    LogisticRegression(random_state=42)
]

def roc_auc_score_cust(y_true, y_hat):
    return roc_auc_score(y_true, y_hat[:,1])

S_train, S_test = stacking(models,
                               x_train, y_train, x_test,
                               regression=False,
                               mode='oof_pred_bag', 
                               needs_proba=True,
                               metric=roc_auc_score_cust,
                               n_folds=5,                
                               stratified=True,          
                               shuffle=True,             
                               random_state=42,         
                               verbose=2)             

last_model = GradientBoostingClassifier(n_estimators=100, max_depth=3,
                                       learning_rate=0.01, 
                                       random_state=42)

last_model.fit(S_train, y_train)
y_hat = last_model.predict_proba(S_test)
roc_auc_score(y_test, y_hat[:, 1])

task:         [classification]
n_classes:    [2]
metric:       [roc_auc_score_cust]
mode:         [oof_pred_bag]
n_models:     [4]

model  0:     [RandomForestClassifier]
    fold  0:  [0.98648743]
    fold  1:  [0.98430279]
    fold  2:  [0.98573038]
    fold  3:  [0.98591965]
    fold  4:  [0.98539918]
    ----
    MEAN:     [0.98556789] + [0.00072452]
    FULL:     [0.98557175]

model  1:     [LogisticRegression]
    fold  0:  [0.99019566]
    fold  1:  [0.99063955]
    fold  2:  [0.98983171]
    fold  3:  [0.98996059]
    fold  4:  [0.98992798]
    ----
    MEAN:     [0.99011110] + [0.00029005]
    FULL:     [0.99011256]

model  2:     [KNeighborsClassifier]


MemoryError: 